<a href="https://colab.research.google.com/github/manavgurnani21/ECS174_Emotion_Detector/blob/GAN-dev/Final_Project_ADM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emotion Detection with Computer Vision

### Overview Add Here

# Dataset Handling

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Loading Dataset

In [5]:
# Initialize Data
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jonathanoheix/face-expression-recognition-dataset")

data_root = os.path.join(path, "images")
train_dir = os.path.join(data_root, "train")
val_dir = os.path.join(data_root, "validation")

## Data Augmentations

### Linear Transformations

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5]), transforms.Grayscale(num_output_channels=1)])

### Synthetic Augmentations

#### Conditional GAN

In [ ]:
# initialize Conditional GAN
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using Apple Silicon GPU (MPS)")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using CUDA GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

# ---------------------
# Hyperparameters
# ---------------------
nz = 100       # noise dimension
ngf = 64       # generator feature maps
ndf = 64       # discriminator feature maps
nc = 1         # image channels
image_size = 48
num_classes = 7
lr_g = 2e-4
lr_d = 2e-4
beta1 = 0.5
num_gan_epochs = 10
batch_size = 64

# ---------------------
# Data transforms
# ---------------------
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*nc, [0.5]*nc)
])

# Assume train_set and test_set are already loaded torchvision datasets
# train_set = ...
# test_set = ...

# Make sure all images are tensors of same size
def make_tensor_dataset(dataset):
    imgs = []
    labels = []
    for img, label in dataset:
        if not isinstance(img, torch.Tensor):
            img = transform(img)
        else:
            # normalize tensor if already tensor
            img = F.interpolate(img.unsqueeze(0), size=(image_size, image_size), mode='bilinear', align_corners=False).squeeze(0)
            img = (img - 0.5)/0.5
        imgs.append(img)
        labels.append(torch.tensor(label))
    imgs = torch.stack(imgs)
    labels = torch.stack(labels)
    return TensorDataset(imgs, labels)

train_dataset = make_tensor_dataset(train_set)
test_dataset = make_tensor_dataset(test_set)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ---------------------
# CGAN Models
# ---------------------
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc, num_classes):
        super().__init__()
        self.label_embed = nn.Embedding(num_classes, num_classes)
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz+num_classes, ngf*8, 3, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        label_embedding = self.label_embed(labels).unsqueeze(2).unsqueeze(3)
        x = torch.cat([noise, label_embedding], dim=1)
        return self.main(x)

class Discriminator(nn.Module):
    def __init__(self, nc, ndf, num_classes):
        super().__init__()
        self.label_embed = nn.Embedding(num_classes, num_classes)
        self.main = nn.Sequential(
            nn.Conv2d(nc+num_classes, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*8, 1, 3, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        B, C, H, W = img.size()
        label_embedding = self.label_embed(labels).unsqueeze(2).unsqueeze(3)
        label_embedding = label_embedding.expand(-1, -1, H, W)
        x = torch.cat([img, label_embedding], dim=1)
        return self.main(x).view(-1)

# ---------------------
# Initialize models and optimizers
# ---------------------
netG = Generator(nz, ngf, nc, num_classes).to(device)
netD = Discriminator(nc, ndf, num_classes).to(device)

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr_d, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr_g, betas=(beta1, 0.999))

# ---------------------
# Training CGAN
# ---------------------
def train_cgan(dataloader, num_epochs=num_gan_epochs):
    netG.train()
    netD.train()
    fixed_noise = torch.randn(64, nz, 1, 1, device=device)
    fixed_labels = torch.randint(0, num_classes, (64,), device=device)

    for epoch in range(num_epochs):
        for i, (real_imgs, labels) in enumerate(dataloader):
            real_imgs = real_imgs.to(device)
            labels = labels.to(device)
            b_size = real_imgs.size(0)

            # Real/Fake labels
            real_label = torch.ones(b_size, device=device)
            fake_label = torch.zeros(b_size, device=device)

            # ------------------
            # Update Discriminator
            # ------------------
            netD.zero_grad()
            # real
            output_real = netD(real_imgs, labels)
            lossD_real = criterion(output_real, real_label)
            # fake
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            random_labels = torch.randint(0, num_classes, (b_size,), device=device)
            fake_imgs = netG(noise, random_labels).detach()
            output_fake = netD(fake_imgs, random_labels)
            lossD_fake = criterion(output_fake, fake_label)

            lossD = lossD_real + lossD_fake
            lossD.backward()
            optimizerD.step()

            # ------------------
            # Update Generator
            # ------------------
            netG.zero_grad()
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            random_labels = torch.randint(0, num_classes, (b_size,), device=device)
            fake_imgs = netG(noise, random_labels)
            output = netD(fake_imgs, random_labels)
            lossG = criterion(output, real_label)
            lossG.backward()
            optimizerG.step()

        print(f"Epoch [{epoch+1}/{num_epochs}] LossD={lossD.item():.4f} LossG={lossG.item():.4f}")

    # Show generated images
    netG.eval()
    with torch.no_grad():
        fake = netG(fixed_noise, fixed_labels).cpu()
    grid = torchvision.utils.make_grid(fake, nrow=8, normalize=True)
    plt.figure(figsize=(8,8))
    plt.axis("off")
    plt.imshow(np.transpose(grid.numpy(), (1,2,0)))
    plt.show()

# ---------------------
# Train CGAN
# ---------------------
train_cgan(train_loader)

# ---------------------
# Generate synthetic dataset
# ---------------------
def generate_synthetic_data(generator, num_samples):
    generator.eval()
    all_imgs = []
    all_labels = []
    with torch.no_grad():
        remaining = num_samples
        while remaining > 0:
            cur_bs = min(batch_size, remaining)
            noise = torch.randn(cur_bs, nz, 1, 1, device=device)
            labels = torch.randint(0, num_classes, (cur_bs,), device=device)
            fake_imgs = generator(noise, labels)
            all_imgs.append(fake_imgs.cpu())
            all_labels.append(labels.cpu())
            remaining -= cur_bs
    all_imgs = torch.cat(all_imgs)
    all_labels = torch.cat(all_labels)
    return TensorDataset(all_imgs, all_labels)

synthetic_dataset = generate_synthetic_data(netG, len(train_dataset))
augmented_dataset = ConcatDataset([train_dataset, synthetic_dataset])
augmented_loader = DataLoader(augmented_dataset, batch_size=batch_size, shuffle=True)

# ---------------------
# Train classifier (ResNet18)
# ---------------------
import torchvision.models as models

resnet18 = models.resnet18(weights=None)
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
resnet18 = resnet18.to(device)

def train_classifier(model, trainloader, testloader, lr=0.001, epochs=20):
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for imgs, labels in trainloader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        print(f"Epoch {epoch+1}/{epochs}, Loss={total_loss/len(trainloader):.4f}, Acc={correct/total:.4f}")

    # Test accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in testloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"Test Accuracy: {correct/total:.4f}")

train_classifier(resnet18, augmented_loader, test_loader)


#### ADA Trained GAN

In [ ]:
# initialize ADA GAN training

## Initializing DataLoaders on Augmented Dataset

In [6]:
train_set = datasets.ImageFolder(root=train_dir, transform=transform)
test_set = datasets.ImageFolder(root=val_dir, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

print("Classes:", train_set.classes)
print("Train size:", len(train_set))
print("Test size:", len(test_set))

Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Train size: 28821
Test size: 7066


## Plotting Training Metrics

In [26]:
def plot_curves(loss, training, test, epochs):
    epochs_axis = range(1, epochs + 1)

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    axes[0].plot(epochs_axis, loss, label='Training Loss')
    axes[0].set_title('Training Loss per Epoch')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(epochs_axis, training, label='Training Accuracy')
    axes[1].set_title('Training Accuracy per Epoch')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].grid(True, alpha=0.3)

    axes[2].plot(epochs_axis, test, label='Test Accuracy')
    axes[2].set_title('Test Accuracy per Epoch')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Accuracy')
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# Single Layer Network

In [7]:
class Single_Layer_Net(nn.Module):
    def __init__(self):
        super(Single_Layer_Net, self).__init__()
        self.fc = nn.Linear(3 * 48 * 48, 10)

    def forward(self, x):
        x = x.view(-1, 3 * 48 * 48)
        x = self.fc(x)
        return x

net = Single_Layer_Net()

In [8]:
def train_model(net, trainloader, testloader, learning_rate = 0.001, criterion = nn.CrossEntropyLoss()):
    epochs = 30
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

    train_losses = []
    train_accs = []
    test_accs = []

    for epoch in range(epochs):
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)

            if isinstance(criterion, nn.MSELoss):
                targets = F.one_hot(labels, num_classes=10).float()
                loss = criterion(outputs, targets)
            else:
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (preds == labels).sum().item()

        avg_loss = running_loss / len(trainloader)
        train_acc = correct_train / total_train

        correct_test = 0
        total_test = 0

        with torch.no_grad():
            for images, labels in testloader:
                outputs = net(images)
                _, predicted = torch.max(outputs, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        test_acc = correct_test / total_test

        train_losses.append(avg_loss)
        train_accs.append(train_acc)
        test_accs.append(test_acc)

    plot_curves(train_losses, train_accs, test_accs, epochs)

In [ ]:
train_model(net, train_loader, test_loader)

### Findings
Both training and testing accuracy were pretty low when using a single layer network. Training accuracy peaked around 42% while testing accuracy fluctuated between about 24% and 37%.

# ResNet18

In [ ]:
class ResidualBlock(nn.Module):
  def __init__(self, input_channels, output_channels, stride=1):
    super(ResidualBlock, self).__init__()
    # 2 convolutional layers in each residual block
    self.conv1 = nn.Conv2d(input_channels, output_channels, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(output_channels)
    self.relu = nn.ReLU(inplace=True)
    self.conv2 = nn.Conv2d(output_channels, output_channels, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(output_channels)
    # setting up skip/shortcut layer
    self.shortcut = nn.Sequential()
    if stride != 1 or input_channels != output_channels:
      self.shortcut = nn.Sequential(nn.Conv2d(input_channels, output_channels, kernel_size=1, stride=stride, bias=False))

  def forward(self, x):
    # x here is input tensor
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    out = out + self.shortcut(x)
    out = self.relu(out)
    return out

In [ ]:
class ResNet18(nn.Module):
  def __init__(self, num_classes=10):
    super(ResNet18, self).__init__()
    self.input_channels = 64
    self.conv_1_x = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.conv_2_x = self.make_layer(ResidualBlock, 64, 2, 1)
    self.conv_3_x = self.make_layer(ResidualBlock, 128, 2, 2)
    self.conv_4_x = self.make_layer(ResidualBlock, 256, 2, 2)
    self.conv_5_x = self.make_layer(ResidualBlock, 512, 2, 2)
    self.bn_1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace=True)
    self.average_pool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc_layer = nn.Linear(512, num_classes)

  def make_layer(self, block, output_channels, num_blocks, stride):
    strides = [stride] + [1] * (num_blocks - 1)
    layers = []
    for stride in strides:
      layers.append(block(self.input_channels, output_channels, stride))
      self.input_channels = output_channels
    return nn.Sequential(*layers)

  def forward(self, x):
    out = self.conv_1_x(x)
    out = self.bn_1(out)
    out = self.relu(out)
    # going through residual blocks of convolutional layers
    out = self.conv_2_x(out)
    out = self.conv_3_x(out)
    out = self.conv_4_x(out)
    out = self.conv_5_x(out)
    # average pooling layer for image condensation
    out = self.average_pool(out)¡
    out = out.view(out.size(0), -1)
    # fully connected layer to get ouputs
    out = self.fc_layer(out)
    return out

model = ResNet18(7)

In [ ]:
train_model(model, train_loader, test_loader)

# ResNet18 with Transfer Learning

# ResNet10

# ResNet10 with Transfer Learning

# Pre-Trained Vision Transformer (ViT) with Transfer Learning